In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model

# ---------------------
# Step 1: Load and Preprocess Images
# ---------------------

base_path = r'D:\Rice-Disease-Simulation\data\train'  # Change this if you're using 'Val/rgb'
img_size = (224, 224)

images = []
labels = []
class_names = sorted(os.listdir(base_path))
label_map = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:
    class_path = os.path.join(base_path, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, img_size)
        images.append(img)
        labels.append(label_map[class_name])

images = np.array(images)
labels = np.array(labels)
images = preprocess_input(images)

print(f"✅ Loaded {len(images)} images across {len(class_names)} classes.")

# ---------------------
# Step 2: Feature Extraction using MobileNetV2
# ---------------------

# Load MobileNetV2 without the top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

# Extract deep features
features = model.predict(images, batch_size=32, verbose=1)
features = features.reshape(features.shape[0], -1)  # Flatten

print(f"✅ Feature shape: {features.shape}")

# ---------------------
# Step 3: Train/Test Split
# ---------------------

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

# ---------------------
# Step 4: Train XGBoost Classifier
# ---------------------

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_estimators=100, max_depth=6)
xgb.fit(X_train, y_train)

# ---------------------
# Step 5: Evaluate
# ---------------------

y_pred = xgb.predict(X_test)

print("\n✅ Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

print("✅ Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

✅ Loaded 2100 images across 6 classes.
66/66 ━━━━━━━━━━━━━━━━━━━━ 15s 213ms/step
✅ Feature shape: (2100, 62720)


C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [16:51:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ Classification Report:

                       precision    recall  f1-score   support

bacterial_leaf_blight       0.96      1.00      0.98        70
           brown_spot       0.87      0.79      0.83        70
              healthy       0.88      0.90      0.89        70
           leaf_blast       0.77      0.77      0.77        70
           leaf_scald       0.95      1.00      0.97        70
    narrow_brown_spot       0.97      0.94      0.96        70

             accuracy                           0.90       420
            macro avg       0.90      0.90      0.90       420
         weighted avg       0.90      0.90      0.90       420

✅ Confusion Matrix:
[[70  0  0  0  0  0]
 [ 3 55  3  8  0  1]
 [ 0  1 63  6  0  0]
 [ 0  7  6 54  2  1]
 [ 0  0  0  0 70  0]
 [ 0  0  0  2  2 66]]


untouched validation code

In [2]:
val_path = r'D:\Rice-Disease-Simulation\data\Validation'  # Path to your Val folder
val_images = []
val_labels = []
img_size = (224, 224)

images = []
labels = []
class_names = sorted(os.listdir(val_path))
label_map = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:  # use same class_names from training
    class_path = os.path.join(val_path, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, img_size)
        val_images.append(img)
        val_labels.append(label_map[class_name])  # use same label_map from training

val_images = np.array(val_images)
val_labels = np.array(val_labels)
val_images = preprocess_input(val_images)

print(f"✅ Loaded {len(val_images)} validation images.")

# Extract features using the same MobileNetV2 model
val_features = model.predict(val_images, batch_size=32, verbose=1)
val_features = val_features.reshape(val_features.shape[0], -1)

# Predict using trained XGBoost model
val_preds = xgb.predict(val_features)

# Evaluate
print("\n✅ Validation Classification Report:\n")
print(classification_report(val_labels, val_preds, target_names=class_names))

print("✅ Validation Confusion Matrix:")
print(confusion_matrix(val_labels, val_preds))

✅ Loaded 528 validation images.
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 254ms/step

✅ Validation Classification Report:

                       precision    recall  f1-score   support

bacterial_leaf_blight       0.97      0.98      0.97        88
           brown_spot       0.89      0.82      0.85        88
              healthy       0.84      0.89      0.86        88
           leaf_blast       0.84      0.82      0.83        88
           leaf_scald       0.96      1.00      0.98        88
    narrow_brown_spot       0.97      0.95      0.96        88

             accuracy                           0.91       528
            macro avg       0.91      0.91      0.91       528
         weighted avg       0.91      0.91      0.91       528

✅ Validation Confusion Matrix:
[[86  0  0  0  2  0]
 [ 1 72  8  5  0  2]
 [ 0  2 78  8  0  0]
 [ 2  5  7 72  1  1]
 [ 0  0  0  0 88  0]
 [ 0  2  0  1  1 84]]
